In [1]:
import vamb

import pandas as pd

import os

In [18]:
BASE_DIR = os.getcwd()

CHECKM_OUTDIR = os.path.join(BASE_DIR, 'example_input_data/new_simulations/camisim_outputs/vamb_outputs/checkm_results')

if not os.path.exists(CHECKM_OUTDIR):
    os.mkdir(CHECKM_OUTDIR)

# How We Will do Benchmarking

# CheckM

One fairly efficient way that we can do benchmarking is to use CheckM which is a package built to support assessment of metagenome binning. 

In [22]:
bins_inpath = os.path.join(BASE_DIR, 'example_input_data/new_simulations/camisim_outputs/vamb_outputs/bins')

#!~/miniconda3/envs/vamb_env/bin/checkm taxonomy_wf -t 32 -x fna genus <taxon> $bins_inpath $CHECKM_OUTDIR
!~/miniconda3/envs/vamb_env/bin/checkm lineage_wf -t 32 -x fna $bins_inpath $CHECKM_OUTDIR

[2021-02-26 12:34:01] INFO: CheckM v1.1.3
[2021-02-26 12:34:01] INFO: checkm lineage_wf -t 32 -x fna /home/pathinformatics/jupyter_projects/vamb/stanford_cs230_project/example_input_data/new_simulations/camisim_outputs/vamb_outputs/bins /home/pathinformatics/jupyter_projects/vamb/stanford_cs230_project/example_input_data/new_simulations/camisim_outputs/vamb_outputs/checkm_results
[2021-02-26 12:34:01] INFO: [CheckM - tree] Placing bins in reference genome tree.
[2021-02-26 12:34:01] INFO: Identifying marker genes in 4 bins with 32 threads:
    Finished processing 4 of 4 (100.00%) bins.
[2021-02-26 12:34:03] INFO: Saving HMM info to file.
[2021-02-26 12:34:03] INFO: Calculating genome statistics for 4 bins with 32 threads:
    Finished processing 4 of 4 (100.00%) bins.
[2021-02-26 12:34:03] INFO: Extracting marker genes to align.
[2021-02-26 12:34:03] INFO: Parsing HMM hits to marker genes:
    Finished parsing hits for 4 of 4 (100.00%) bins.
[2021-02-26 12:34:03] INFO: Extracting 43 HM

When CheckM runs, it creates several reports that we can use to ultimately assess quality. We can use the qa command to expand upon these

# Custom

### Recall
In order to effectively benchmark these binning exercises, we need to understand precision and recall. Recall in this case would be the number of basepairs in a given genome covered by the bin for that genome:

$$\Large Recall = \frac{bases_{genome} \bigcap (  \bigcup_{i=1}^{n} bases_{contig^i} )} {bases_{genome} \bigcap  (\bigcup_{i=1}^{all contigs} bases_{contig^i} )} $$

Where $n$ is the number of contigs in a given bin of contigs


### Precision

While the precision for a given bin would be the number of bases of a genome represented in a bin as a proportion of the number of bases in any genome that are captured in that bin:


$$\Large Precision = \frac{ \bigcup_{j=1}^{K}( bases_{genome}) \bigcap (  \bigcup_{i=1}^{n} bases_{contig^i} )} {bases_{genome} \bigcap  (\bigcup_{i=1}^{n} bases_{contig^i} )} $$

Where $n$ is the number of contigs in a given bin of contigs and $K$ is the number of genomes

In [3]:
# First load in the Reference
reference_path = os.path.join(BASE_DIR, 'benchmarking_data/airways/reference.tsv')

!head $reference_path # show first 10 lines of reference file

with open(reference_path) as reference_file:
    reference = vamb.benchmark.Reference.from_file(reference_file)

S23C236194	OTU_97.38699.0	CP013953.1	405835	408637
S23C27593	OTU_97.44406.0	CP015430.1	1591817	1594726
S23C53437	OTU_97.35057.0	CP012974.1	260542	266700
S23C191539	OTU_97.16597.0	CP010116.1	1675948	1678686
S23C111210	OTU_97.20680.0	CP013619.1	408162	411064
S23C216129	OTU_97.409.0	CP013957.1	103779	106015
S23C2782	OTU_97.45246.0	CP007573.1	1941619	1956892
S23C9729	OTU_97.44851.0	CP015426.1	1099587	1121671
S23C36165	OTU_97.409.0	CP013957.1	427732	429801
S23C90379	OTU_97.2104.0	CP019196.1	300725	303071


In [4]:
taxonomy_path = os.path.join(BASE_DIR, 'benchmarking_data/airways/taxonomy.tsv')

!head $taxonomy_path # show first 10 lines of reference file

with open(taxonomy_path) as taxonomy_file:
    reference.load_tax_file(taxonomy_file)

OTU_97.39766.0	Corynebacterium pseudotuberculosis	Corynebacterium
OTU_97.7195.0	Acinetobacter baumannii	Acinetobacter
OTU_97.34725.0	Staphylococcus aureus	Staphylococcus
OTU_97.1051.0	Corynebacterium ulcerans	Corynebacterium
OTU_97.34663.0	Staphylococcus aureus	Staphylococcus
OTU_97.8581.0	Corynebacterium doosanense	Corynebacterium
OTU_97.8702.0	Flavobacterium psychrophilum	Flavobacterium
OTU_97.31374.1	Corynebacterium ulcerans	Corynebacterium
OTU_97.479.0	Flavobacterium gilvum	Flavobacterium
OTU_97.37290.0	Staphylococcus equorum	Staphylococcus


In [5]:
with open( os.path.join(BASE_DIR, 'example_input_data/new_simulations/camisim_outputs/vamb_outputs/clusters.tsv') ) as clusters_file:
    vamb_clusters = vamb.vambtools.read_clusters(clusters_file)
    vamb_bins = vamb.benchmark.Binning(vamb_clusters, reference, minsize=200000)

KeyError: 'Contigname S0C3223 not in reference'